In [ ]:
import segmentation_models_pytorch as smp
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/unetMobileNetTerrenator_0.001Grafica')
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else b"cpu"#mps stands for matrix product states, basically enables high-performance training
print("Using: ", device)

**Creating model**

In [ ]:
model = smp.Unet(encoder_name="mobilenet_v2", encoder_weights=None, classes=2, activation='softmax')
#model.classification_head = torch.nn.Conv2d(512, 2, kernel_size=1, stride=1)
model.to(device)
print(model)

In [ ]:
for count,child in enumerate(model.children()):
    print(" Child ", count , "is -")
    print(child)

In [ ]:
'''#First 3 layers are freezed
for parents in model.children():
   
   for count,child in enumerate(parents.children()):
      
      if count==3:
         break
      for param in child.parameters():
         param.requires_grad=False
      print("Child ",count," is frozen now")
      print(child)'''

In [ ]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn

preprocess_input = get_preprocessing_fn('mobilenet_v2', pretrained="imagenet")

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
img = Image.open("Img2.png")
img = preprocess_input(np.array(img))
plt.imshow(img)

**DATASET**

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.transforms import Lambda
import torchvision.datasets as datasets
from PIL import Image
import os
import pandas as pd
import numpy as np
print(torch.__version__)

*****

In [ ]:
folder1 = "SemanticSegmentationFotos/all/masks"
folder2 = "SemanticSegmentationFotos/all/images"


if not os.path.exists(folder2):
    os.makedirs(folder2)

In [ ]:
dir1 = "SemanticSegmentationFotos/AntihorarioMasks"
dir2 = "SemanticSegmentationFotos/fotosCarrilAntihorario"

for root,dirs,files in os.walk(dir1):
    for file in files:
        for root2,dirs2,files2 in os.walk(dir2):
            if file in files2:
                rutaMask = folder1 +"/"+file
                rutaImagen = folder2 +"/"+file
                imgMask = Image.open(dir1+"/"+file)
                imgMask.save(rutaMask)
                img = Image.open(dir2+"/"+file)
                img.save(rutaImagen)


In [ ]:
dir1 = "SemanticSegmentationFotos/Antihorario2Masks"
dir2 = "SemanticSegmentationFotos/fotosCarrilAntihorario2"

for root,dirs,files in os.walk(dir1):
    for file in files:
        for root2,dirs2,files2 in os.walk(dir2):
            if file in files2:
                rutaMask = folder1 +"/"+file.split(".")[0]+"v2.png"
                rutaImagen = folder2 +"/"+file.split(".")[0]+"v2.png"
                imgMask = Image.open(dir1+"/"+file)
                imgMask.save(rutaMask)
                img = Image.open(dir2+"/"+file)
                img.save(rutaImagen)

****

In [ ]:
import glob
folder_data = glob.glob("SemanticSegmentationFotos/all/images/*.png")
folder_mask = glob.glob("SemanticSegmentationFotos/all/masks/*.png")

In [ ]:
len_data = len(folder_data)
print(len_data)
train_size = 0.8

In [ ]:
train_image_paths = folder_data[:int(len_data*train_size)]
test_image_paths = folder_data[int(len_data*train_size):]

train_mask_paths = folder_mask[:int(len_data*train_size)]
test_mask_paths = folder_mask[int(len_data*train_size):]


In [ ]:
from torch.utils.data.dataset import Dataset
class UnetDataset(Dataset):
    def __init__(self, image_paths, target_paths, transform1 = None, transform2 = None, train=True):   # initial logic 
        self.image_paths = image_paths
        self.target_paths = target_paths
        self.transform1 = transform1
        self.transform2 = transform2

    def __getitem__(self, index):

        image = Image.open(self.image_paths[index])
        #image.show()
        mask = Image.open(self.target_paths[index])
        #mask.show()
        if self.transform1:
            t_image = self.transform1(image)
        if self.transform2:
            t_mask = self.transform2(mask)
        return t_image, t_mask

    def __len__(self):  # return count of sample we have

        return len(self.image_paths)

**Dataloader**

In [ ]:
import cv2
class Preprocessing1:
    def __init__(self, porcentaje_cropped):
        self.porcentaje = porcentaje_cropped

    def __call__(self, x):
        img_array = np.asarray(x)
        img_array = preprocess_input(np.array(img_array))
        W, H, C = img_array.shape
        img = img_array[int(self.porcentaje*W):,:,:]
        img = cv2.resize(img, (256, 256))
        return img
class Preprocessing2:
    def __init__(self, porcentaje_cropped):
        self.porcentaje = porcentaje_cropped

    def __call__(self, x):
        img_array = np.asarray(x)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        
        W, H= img_array.shape
        img = img_array[int(self.porcentaje*W):,:]
        img = cv2.resize(img, (256, 256))
        return img
class ToFloat32:
    def __call__(self, x):
        return x.to(torch.float32)
transform1 = transforms.Compose([
    # you can add other transformations in this list
    Preprocessing1(porcentaje_cropped = 0.5),
    transforms.ToTensor(),
    ToFloat32(),
])
transform2 = transforms.Compose([
    # you can add other transformations in this list
    Preprocessing2(porcentaje_cropped = 0.5),
    transforms.ToTensor(),
    ToFloat32(),
])
train_dataset = UnetDataset(train_image_paths, train_mask_paths, transform1, transform2, train=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = UnetDataset(test_image_paths, test_mask_paths, transform1, transform2, train=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

**Visualizing**

In [ ]:
fig=plt.figure(0, (10,6))
for x , y in train_loader:#Each element x and y (from iterable "test_dataloader") is a batch of 'batch_size' features and labels
    print(x.shape, y.shape)
    for i in range(8):
        fig.add_subplot(4,2,i+1)
        #plt.imshow(np.transpose(x[i], (1, 2, 0)))#COnvertir al formato de matplotlib
        plt.imshow(np.transpose(y[i], (1, 2, 0)))#COnvertir al formato de matplotlib
        #plt.title("Direccion: "+str(int(y[i])))
    print(x[0].dtype)
    print("Shape of x [N, C, H, W]: ",x.shape)#tiene dimensiones 32(batch_size)xC(n_channels)xH(Height)xW(Width)
    print("Shape of y: ",y.shape)#tiene dimensiones 32(batch_size), pues para cada elemento del batch hay un label
    break

Optim

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()#Hace el softmax y la funcion costo al mismo tiempo
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Train

In [ ]:
def train(dataloader, model, loss_function, optimizer, epoch):
    train_correct_num, train_total, train_cost_acum = 0, 0, 0.0
    size = len(dataloader.dataset)
    model.train()
    for batch, (x,y) in enumerate(dataloader):
        x = x.to(device=device, dtype=torch.float32)
        y = y.to(device=device, dtype=torch.long).squeeze(1)
        print(x.shape)
        scores = model(x)
        cost = loss_function(input=scores, target=y)
        #cost.requires_grad = True

        #Añadir a la grafica
        writer.add_scalar("Loss/train", cost, epoch)

        #Backpropagation
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        train_predictions = torch.argmax(scores, dim=1)
        train_correct_num += (train_predictions == y).sum()
        train_total += torch.numel(train_predictions)#total numero de elementos de train_predictions, if size=[a,b,c ] returns a*b*c
        train_cost_acum += cost
        
        if batch % 9 == 1:
            train_acc = float(train_correct_num)/train_total#el train total ya considera el batch
            train_cost_every = float(train_cost_acum)/batch#Se divide entre el batch, xq por cada batch se aumenta el loss
            writer.add_scalar('training loss', train_cost_every, epoch*size+batch)
            writer.add_scalar('training accuracy', train_acc, epoch*size+batch)
            print("loss: ", train_cost_every, "Accuracy: ", train_acc)

In [ ]:
def test(dataloader, model, loss_function,t):
    num_batches = len(dataloader)
    model.eval()
    test_loss, train_correct_num, train_total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device=device, dtype=torch.float32)
            y = y.to(device=device, dtype=torch.long).squeeze(1)
            pred = model(x)
            loss = loss_function(input=pred, target=y)
            
            test_loss += loss.item()
            train_predictions = torch.argmax(pred, dim=1)
            train_correct_num += (train_predictions == y).sum()
            train_total += torch.numel(train_predictions)

    test_loss /= num_batches
    train_correct_num = train_correct_num/train_total
    writer.add_scalar('test accuracy', train_correct_num*100, t)
    print(f"Test Error: \n Accuracy: {(100*train_correct_num):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 25
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer, t)
    test(test_loader, model, loss_fn, t)
writer.flush()

In [ ]:
torch.save(model.state_dict(), "unetMobilenetv2WithoutDataAug.pth")

In [ ]:
modelCharged = smp.Unet(encoder_name="mobilenet_v2", encoder_weights=None, classes=2, activation='softmax')
modelCharged.load_state_dict(torch.load("unetMobilenetv2WithoutDataAug.pth"))
modelCharged.eval()

In [ ]:
modelCharged = torch.load("AlvaroUnetv1.pth")
ONNX_FILE_PATH = 'AlvaroUnetv1.onnx'
input = torch.rand(8, 3, 256, 256)
torch.onnx.export(modelCharged, input, ONNX_FILE_PATH, input_names=['input'],
                  output_names=['output'], export_params=True)

In [ ]:
import onnx
onnx_model = onnx.load(ONNX_FILE_PATH)
onnx.checker.check_model(onnx_model)

In [ ]:
torch.save(modelCharged, "unetMobilenetv2WithoutDataAugEntire.pth")

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import time
model.eval()
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

img = transform1(Image.open("Img509.png"))
ax[0].imshow(img.permute(1, 2, 0))
ax[0].set_title('Figura 1')
img = img.to("cuda:0")
#plt.imshow(  img.permute(1, 2, 0)  )
#print(img.shape)
#Image.fromarray(img.numpy()).show()
model.to("cuda:0")
with torch.no_grad():
    a=time.time()
    output = model(img.unsqueeze(0))
    output = output.to("cpu")
    maskOut =torch.argmax(output,1).numpy()
    print(time.time()-a)
    ax[1].imshow(maskOut[0])
    ax[1].set_title('Figura 2')
    print(maskOut.shape)
    a=maskOut.astype('uint8')*255
    print(a.shape)
    #imagen = Image.fromarray(a[0])
    #imagen.show()

In [ ]:
model.to("cuda:0")
torch.save(model, "unetEntireModel.pth")

In [ ]:
torch.save(model.state_dict(), "unetModelParte.pth")

In [ ]:
import time
modeloCargado = torch.load("unetEntireModel.pth")
img = transform1(Image.open("Img509.png"))
print(img.is_cuda)
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(img.permute(1, 2, 0))
ax[0].set_title('Figura 1')
img = img.to("cuda:0")
print(img.is_cuda)
#plt.imshow(  img.permute(1, 2, 0)  )
#print(img.shape)
#Image.fromarray(img.numpy()).show()
modeloCargado.to("cuda:0")
with torch.no_grad():
    a=time.time()
    output = modeloCargado(img.unsqueeze(0))
    print(time.time()-a)
    a = output[0].to("cpu")
    maskOut =torch.argmax(a,1).numpy()
    ax[1].imshow(maskOut[0])
    ax[1].set_title('Figura 2')
    print(maskOut.shape)

In [ ]:
modeloCargado = torch.load("unetEntireModel.pth")
img = transform1(Image.open("Img25.png"))
print(img.is_cuda)
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(img.permute(1, 2, 0))
ax[0].set_title('Figura 1')
img = img.to("cpu")
print(img.is_cuda)
#plt.imshow(  img.permute(1, 2, 0)  )
#print(img.shape)
#Image.fromarray(img.numpy()).show()
modeloCargado.to("cpu")
with torch.no_grad():
    a=time.time()
    output = modeloCargado(img.unsqueeze(0))
    print(time.time()-a)
    a = output[0].to("cpu")
    maskOut =torch.argmax(a,1).numpy()
    ax[1].imshow(maskOut[0])
    ax[1].set_title('Figura 2')
    print(maskOut.shape)

In [ ]:
import time
import segmentation_models_pytorch as smp
modeloCargado = smp.Unet(encoder_name="resnet34", classes=2, activation='softmax')
modeloCargado.classification_head = torch.nn.Conv2d(512, 2, kernel_size=1, stride=1)
modeloCargado.load_state_dict(torch.load("unetModelParte.pth"))
model.eval()
img = transform1(Image.open("Img147.png"))
print(img.is_cuda)
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(img.permute(1, 2, 0))
ax[0].set_title('Figura 1')
img = img.to("cuda:0")
print(img.is_cuda)
#plt.imshow(  img.permute(1, 2, 0)  )
#print(img.shape)
#Image.fromarray(img.numpy()).show()
modeloCargado.to("cuda:0")
with torch.no_grad():
    a=time.time()
    output = modeloCargado(img.unsqueeze(0))
    print(time.time()-a)
    a = output[0].to("cpu")
    maskOut =torch.argmax(a,1).numpy()
    ax[1].imshow(maskOut[0])
    ax[1].set_title('Figura 2')
    print(maskOut.shape)